# Data Importing and Preprocessing - Women's March 2017 dataset

Este notebook trata da parte de leitura dos dados dos tweets de arquivos `.json` e pré-processamento das informações contidas nestes arquivos.

Quando a API do Twitter retorna dados de tweets recuperados através dela, eles são entregues em um arquivo JSON com vários dados e metadados que compõe um tweet. Muitos desses dados contidos no JSON são desnecessários para a análise que temos como objetivo, por isso, os trechos de código abaixo tratam as informações necessárias para a análise e descartam as demais.

Para esse trabalho utilizamos bibliotecas específicas além do que a linguagem provém:

- **`os`**: lida com detalhes específicos ao sistema operacional da máquina rodando a análise, independente de qual sistemaoperacional seja;
- **`re`**: lida com expressões regulares pra reconhecimento de padrões em textos;
- **`glob`**: lida com padrão de arquivos em sistemas Unix;
- **`pandas`**: toolkit para análise de dados;
- **`json_normalize`**: um sub-módulo do pandas para tratamento de arquivos e dados JSON.

In [1]:
import os
import re
import glob
import pandas as pd
from pandas.io.json import json_normalize

Com auxílio da biblioteca **`re`**, foram criadas funções para converter algumas informações presentes no texto do tweet para informações simples e concisas para análise posterior.

In [2]:
# Helper functions
def get_hashtags(text):
    s = re.findall('(?:^|\s)[＃#]{1}(\w+)', text)
    return s if len(s) > 0 else ''

def get_mentions(text):
    s = re.findall('(?:^|\s?|\.?)[＠@]{1}([^\s#<>[\]|{}:;,.\(\)=+]+)', text)
    return s if len(s) > 0 else ''

def get_source(text):
    s = re.findall('<a\s+?href=\"[^\"]*\"\s+?rel=\"[^\"]*\">([^<>]+)<\/a>', text)
    return s[0] if len(s) > 0 else ''

def get_urls(text):
    s = re.findall('http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', text)
    return s[0] if len(s) > 0 else ''

In [3]:
path = '../data/'
filenames = glob.glob(os.path.join(path, '*.json'))
filenames.sort()

Devido ao enorme número de dados que foram recuperados da API (11.249.944 tweets), o pré-processamento foi feito em blocos. 

Na importação dos dados da API, os tweets foram divididos em 16 arquivos, e na importação dos dados esses 16 arquivos foram lidos, pré-processados, e outros 16 arquivos foram criados com os dados prontos para a análise, que geraram 9.170.486 instâncias.

Os dados para análise se resumem em:

- id do tweet
- data e hora de criação do tweet
- dispositivo fonte
- texto do tweet
- hashtags presentes no tweet
- menções à usuários
- urls presentes no tweet
- número de vezes que o tweet foi favoritado
- número de vezes que o tweet foi retweetado
- localização do usuário
- nome do usuário
- username
- quantos seguidores o usuário tem
- se é um usuário verificado ou não

Após o pré-processamento dos tweets, onde foram escolhidos apenas 14 características para observação, conseguimos reduzir a nossa amostra de um arquivo de cerca de 96GB iniciais para um arquivo de 4,8GB, descartando apenas os metadados desnecessários e os tweets de idioma diferente do inglês.

In [4]:
for file in filenames:
    json_reader = pd.read_json(file, lines=True, chunksize=2048)

    wm_data = pd.DataFrame()

    for chunk in json_reader:
        not_truncated = chunk[chunk['truncated'] == False]
        only_english = not_truncated[not_truncated['lang'] == 'en'].reset_index()

        only_english['hashtags'] = only_english['text'].apply(get_hashtags)
        only_english['mentions'] = only_english['text'].apply(get_mentions)
        only_english['urls'] = only_english['text'].apply(get_urls)
        only_english['source'] = only_english['source'].apply(get_source)
        user_df = json_normalize(only_english['user'])

        # Selecting only few columns
        tweet_df = only_english[['id_str', 'created_at', 'source', 'text', 'hashtags', 'mentions', \
                                 'urls', 'favorite_count', 'retweet_count']]
        user_df = user_df[['location', 'name', 'screen_name', 'followers_count', 'verified']]

        frames = [tweet_df, user_df]

        df = pd.concat(frames, axis=1)

        wm_data.append(df)
        
    fp = file[2:]
    filepath = '../data/clean_{}'.format(fp)    
        
    with open(filepath, 'w') as f:
        f.write(wm_data.to_json(orient='records', lines=True))